<a href="https://colab.research.google.com/github/raul27868/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/SEMINARIO/Seminario(plantilla).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
#### Douglas Humberto Acosta Egas:   <br>

# Url: PONER URL <br>

Problema:
> 1. Elección de grupos de población homogéneos <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Combinar cifras y operaciones

Descripción del problema:(copiar enunciado)

....

<font color='red'>(*) La respuesta es obligatoria</font>





                                        

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Problema 1: Elección de grupos de población homogéneos 

## Descripcion

Una productora ganadera nos encarga la tarea de seleccionar grupos de terneros para aplicar 3 tratamientos diferentes. Para cada uno de los tratamientos debemos seleccionar 3 grupos de terneros que sean lo mas homogéneos posible en peso para que en los  resultados del tratamiento influya lo menos posible el peso del animal. Disponemos de una población de N animales entre machos y hembras

### Restricciones

Aun se desconocen los datos concretos de los animales por lo que debemos trabajar con datos que debemos generar de forma aleatoria. Lo que si nos proporcionan es una plantilla de los datos:

ID | Peso | Sexo
-- | -- | --
1 | 38.7 | M
2 | 37.1 | H
3 | 39.2 | M

Hemos podido averiguar que el peso medio del ternero es 37kg con una desviación estándar de 2.1

<font color="red"> Se solicita diseñar un algoritmo para conseguir una agrupación que cumpla de la mejor manera posible las especificaciones de la productora. </font>

<font color="red">(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br></font>

#### Respuesta:

La pregunta sin restricciones sería: "Una productora ganadera nos encarga la tarea de seleccionar grupos de terneros para aplicar 3 tratamientos diferentes"

La solución que buscamos en este caso es cuántos grupos podemos generar en una población de N individuos.

$$ \sum_{i=1}^{n} \binom{n}{k} $$

En combinatoria, especificamente el análisis de particiones, existe "el número de Stirling" que analiza la cantidad de particiones en las que una población se puede dividir.  Pienso que esta puede ser otra respuesta al problema de analizar cuántos grupos podemos generar de una población, salvo que este análisis aplica la restricción de que ningun individuo queda fuera de los grupos.

https://en.wikipedia.org/wiki/Stirling_numbers_of_the_second_kind

<font color="red">¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.</font>

#### Respuesta




<font color="red"> Modelo para el espacio de soluciones<br> </font>

<font color="red">(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)</font>

#### Respuesta

Usé un dataframe porque me permite ordenar, recorrerlo usando un "for" y hacer calculos estadisticos basicos (media, desviacion estandar) de forma sencilla.

Pude haber usado diccionarios para asignar los individuos a los grupos y tratamientos, pero considero que esto sería menos eficiente pues tendria al menos dos diccionarios, uno para asignar los individuos a los grupos y otro para asignar los grupos a los tratamientos.

<font color="red">
    Según el modelo para el espacio de soluciones<br>
    (*)¿Cual es la función objetivo?
</font>

### Respuesta

La función objetivo es "tener tres grupos homogeneos para cada tratamiento", la homogeneidad se determina por el peso (media y desviacion estandar) y por la cantidad de individuos.

El objetivo es minimizar la desviacion estandar de los pesos promedios dentro de cada grupo y dentro de cada tratamiento.


#### planificación del desarrollo

- Primero voy a definir la población de "N" terneros, generando datos aleatorios utilizando la media del peso y su desviacion estandar:
    - Peso: media de 37 y desviacion estándar de 2.1
    - Sexo: selección aleatoria con probabilidad 50/50 de que un ternero sea Macho
- Luego voy a recorrer la lista, asignando a cada ternero el grupo y tratamiento que le corresponde utilizando como referencia su "identificación" (indice).


In [2]:
# Con el fin de aplicar el experimento en distintos escenarios
def genera_poblacion_aleatoria(terneros):
    
    #generacion de datos aleatorios
    #terneros = np.random.randint(300,1000)
    
    #print("terneros: ", terneros)
    array_sexo = ["H", "M"]

    #lista para el indice de terneros que empiece en "1"
    lista_indice =  list(range(1, terneros+1))

    #lista del sexo de cada individuo
    lista_sexo = list(np.random.choice(array_sexo, terneros, p=[0.5, 0.5]))

    #lista del peso de cada individuo
    lista_peso = list( np.around( np.random.normal( 37, 2.1, terneros ), 1 ) )

    #dataframe con los datos aleatorios de los individuos
    df_terneros = pd.DataFrame({"Peso":lista_peso, "Sexo": lista_sexo}, index=lista_indice)
    #print("la media de los " + str(terneros) + " es: " + str(round(df_terneros.mean(),1)))
    #print("la desviacion estandar de los " + str(terneros) + " es: " + str(round(df_terneros.std(),1)))
    return df_terneros

In [3]:
#esta funcion divide a la poblacion en 9 grupos
def dividir_poblacion_9 ( df_poblacion ) :
    print("\r\nEXPERIMENTO\r\n")
    # almaceno la cantidad de terneros que tendrá los dos primeros grupos, el tercer grupo tendrá todos los terneros que sobren
    # en poblaciones divisibles por 3 los grupos tendrán la misma cantidad de invididuos
    cantidad_terneros = int( df_poblacion.shape[0] / 9 )
    grupo = 0
    tratamiento = 0
    print("cantidad de terneros totales: ", len(df_poblacion))
    print("cantidad de terneros por grupo: ", cantidad_terneros)
    
    #realizo un ordenamiento por peso para reducir la desviacion estandar dentro de cada grupo
    #almaceno en un dataframe para no dañar el original
    df_poblacion_ordenada = df_poblacion.sort_values( by = ['Peso'] )    
    
    #realizo una reindexacion a toda la poblacion para extraer los grupos
    nuevo_indice =  list(range(1, df_poblacion_ordenada.shape[0]+1))
    df_poblacion_ordenada.index = nuevo_indice
    #display(df_poblacion_ordenada)

    #creo una nueva columna en el dataframe para almacenar el grupo
    df_poblacion_ordenada["Grupo"] = ""
    df_poblacion_ordenada["Tratamiento"] = ""
    
    for i in range(1, df_poblacion_ordenada.shape[0]+1 ) :
        
        #el grupo es una asignacion secuencial
        if ( ( i - 1 ) % cantidad_terneros == 0 ) :
            grupo += 1
        
        if( grupo > 9 ):
            grupo = 9
        
        # el tratamiento es una asignacion que mezcla los grupos iniciales para aumentar la homogeneidad
        # en la aplicacion de los tratamientos
        tratamiento = ( ( grupo - 1 ) % 3 ) + 1
            
        grupo_asignado = "Grupo " + str(grupo)
            
        df_poblacion_ordenada.loc[ i, "Grupo" ] = grupo_asignado

    return df_poblacion_ordenada

In [4]:
#una funcion que calcule la poblacion aleatoria y presente los resultados
def experimento(terneros):
    df_terneros = genera_poblacion_aleatoria(terneros)

    solucion = dividir_poblacion_9(df_terneros)
    
    # asignacion ordenada de tratamientos, pienso que esta es la MEJOR SOLUCION porque 
    # dentro de cada grupo tiene la menor desviacion de pesos
    for i in range(1, solucion.shape[0] + 1 ) :
        #print(solucion.loc[ i, "Grupo" ][-1])
        tratamiento = int((int(solucion.loc[ i, "Grupo" ][-1])-1) / 3)+1
        solucion.loc[ i, "Tratamiento" ] = "T" + str(tratamiento)

    #df_grupos = pd.pivot_table(solucion, values="Peso", index=["Tratamiento", "Grupo"], aggfunc={'Peso': [np.mean, np.std, np.count_nonzero]}).round(1)
    sol_tratamientos = solucion.groupby(["Tratamiento", "Grupo"], as_index=False)["Peso"].agg({"PesoStd": "std", "PesoProm":"mean"})

    soluciones = pd.DataFrame(columns=["T1", "dT1", "T2", "dT2", "T3", "dT3", "dTotal"], data=[["1,2,3", 0.0, "4,5,6", 0.0, "7,8,9", 0.0, 0.0]], index=[[0]])
    soluciones.loc[0,"dT1"] = round( sol_tratamientos[sol_tratamientos["Tratamiento"]=="T1"]["PesoProm"].std(), 1)
    soluciones.loc[0,"dT2"] = round( sol_tratamientos[sol_tratamientos["Tratamiento"]=="T2"]["PesoProm"].std(), 1)
    soluciones.loc[0,"dT3"] = round( sol_tratamientos[sol_tratamientos["Tratamiento"]=="T3"]["PesoProm"].std(), 1)
    soluciones.loc[0,"dTotal"] = soluciones.loc[0,"dT1"] + soluciones.loc[0,"dT2"] + soluciones.loc[0,"dT3"]

    #Buscar la mejor solucion FUERZA BRUTA
    for i in range(1, 1000):
        sol_tratamientos = sol_tratamientos.sample(frac=1).reset_index(drop=True)
        
        for j in range(0, sol_tratamientos.shape[0] ) :
            tratamiento = int((sol_tratamientos.index[ j ] / 3)+1)
            sol_tratamientos.loc[ j, "Tratamiento" ] = "T" + str(tratamiento)
            
        nueva_solucion = pd.DataFrame(columns=["T1", "dT1", "T2", "dT2", "T3", "dT3", "dTotal"], data=[["1,2,3", 0.0, "4,5,6", 0.0, "7,8,9", 0.0, 0.0]], index=[[i]])
        nueva_solucion.loc[i, "T1"] = ",".join([s.replace('Grupo ' , '') for s in sol_tratamientos[sol_tratamientos["Tratamiento"]=="T1"]["Grupo"].values ])
        nueva_solucion.loc[i,"dT1"] = round( sol_tratamientos[sol_tratamientos["Tratamiento"]=="T1"]["PesoProm"].std(), 1)
        
        nueva_solucion.loc[i, "T2"] = ",".join([s.replace('Grupo ' , '') for s in sol_tratamientos[sol_tratamientos["Tratamiento"]=="T2"]["Grupo"].values ])
        nueva_solucion.loc[i,"dT2"] = round( sol_tratamientos[sol_tratamientos["Tratamiento"]=="T2"]["PesoProm"].std(), 1)
        
        nueva_solucion.loc[i, "T3"] = ",".join([s.replace('Grupo ' , '') for s in sol_tratamientos[sol_tratamientos["Tratamiento"]=="T3"]["Grupo"].values ])
        nueva_solucion.loc[i,"dT3"] = round( sol_tratamientos[sol_tratamientos["Tratamiento"]=="T3"]["PesoProm"].std(), 1)
        
        nueva_solucion.loc[i,"dTotal"] = nueva_solucion.loc[i,"dT1"] + nueva_solucion.loc[i,"dT2"] + nueva_solucion.loc[i,"dT3"]
        soluciones = pd.concat([soluciones, nueva_solucion])

    #ordenar las soluciones de forma ascendente
    soluciones = soluciones.sort_values( by = ['dTotal'] )    
    display(soluciones.head(10))
    #map(lambda s: s.replace('HD\,' , 'HD '), L)
    #display( ",".join([s.replace('Grupo ' , '') for s in sol_tratamientos[sol_tratamientos["Tratamiento"]=="T1"]["Grupo"].values ]))
    
    

In [5]:
experimento(500)


EXPERIMENTO

cantidad de terneros totales:  500
cantidad de terneros por grupo:  55


,T1,dT1,T2,dT2,T3,dT3,dTotal
0,"1,2,3",1.2,"4,5,6",0.6,"7,8,9",1.1,2.9
38,"1,3,2",1.2,"5,4,6",0.6,"8,9,7",1.1,2.9
54,"7,9,8",1.1,"3,1,2",1.2,"6,4,5",0.6,2.9
928,"9,8,7",1.1,"3,2,1",1.2,"5,6,4",0.6,2.9
306,"3,1,2",1.2,"9,8,7",1.1,"5,6,4",0.6,2.9
42,"8,7,9",1.1,"6,4,5",0.6,"1,2,3",1.2,2.9
234,"4,6,5",0.6,"8,7,9",1.1,"1,2,3",1.2,2.9
954,"8,7,9",1.1,"6,5,4",0.6,"2,1,3",1.2,2.9
259,"3,1,2",1.2,"6,8,9",1.4,"4,5,7",0.9,3.5
28,"6,8,9",1.4,"1,3,2",1.2,"5,4,7",0.9,3.5


<font color="red">(*)¿Es un problema de maximización o minimización?</font>

### Respuesta

Es un problema de minimización, puesto que la tarea es hacer que los grupos sean lo más homogeneos posibles, una de las formas de lograrlo es minimizando la desviacion estandar de los grupos dentro de cada tratamiento.

<font color="blue">Diseña un algoritmo para resolver el problema por fuerza bruta</font>

#### Respuesta

In [6]:
#una funcion que calcule la poblacion aleatoria este es un dato del problema
df_terneros = genera_poblacion_aleatoria(600)

def solucionFuerzaBruta(iteraciones):

    # ordena la lista y asigna el grupo del 1 al 9 en grupos de igual tamaño
    # el sobrante de individuos se asigna al grupo final
    solucion = dividir_poblacion_9(df_terneros)

    # recolecto los resultados en un dataframe para evaluar la mejor solucion al final de las iteraciones
    sol_tratamientos = solucion.groupby(["Tratamiento", "Grupo"], as_index=False)["Peso"].agg({"PesoStd": "std", "PesoProm":"mean"})
    soluciones = pd.DataFrame(columns=["T1", "dT1", "T2", "dT2", "T3", "dT3", "dTotal"])
    
    #Buscar la mejor solucion FUERZA BRUTA
    # mezclar aleatoriamente los grupos para cada tratamiento
    for i in range(0, iteraciones):
        sol_tratamientos = sol_tratamientos.sample(frac=1).reset_index(drop=True)
        
        for j in range(0, sol_tratamientos.shape[0] ) :
            tratamiento = int((sol_tratamientos.index[ j ] / 3)+1)
            sol_tratamientos.loc[ j, "Tratamiento" ] = "T" + str(tratamiento)
            
        nueva_solucion = pd.DataFrame(columns=["T1", "dT1", "T2", "dT2", "T3", "dT3", "dTotal"], index=[i])
        nueva_solucion.loc[i, "T1"] = ",".join([s.replace('Grupo ' , '') for s in sol_tratamientos[sol_tratamientos["Tratamiento"]=="T1"]["Grupo"].values ])
        nueva_solucion.loc[i,"dT1"] = round( sol_tratamientos[sol_tratamientos["Tratamiento"]=="T1"]["PesoProm"].std(), 2)
        
        nueva_solucion.loc[i, "T2"] = ",".join([s.replace('Grupo ' , '') for s in sol_tratamientos[sol_tratamientos["Tratamiento"]=="T2"]["Grupo"].values ])
        nueva_solucion.loc[i,"dT2"] = round( sol_tratamientos[sol_tratamientos["Tratamiento"]=="T2"]["PesoProm"].std(), 2)
        
        nueva_solucion.loc[i, "T3"] = ",".join([s.replace('Grupo ' , '') for s in sol_tratamientos[sol_tratamientos["Tratamiento"]=="T3"]["Grupo"].values ])
        nueva_solucion.loc[i,"dT3"] = round( sol_tratamientos[sol_tratamientos["Tratamiento"]=="T3"]["PesoProm"].std(), 2)
        
        nueva_solucion.loc[i,"dTotal"] = nueva_solucion.loc[i,"dT1"] + nueva_solucion.loc[i,"dT2"] + nueva_solucion.loc[i,"dT3"]
        soluciones = pd.concat([soluciones, nueva_solucion])

    #ordenar las soluciones de forma ascendente
    soluciones = soluciones.sort_values( by = ['dTotal'] )
    #al inicio del dataframe estarán las soluciones encontradas, el primero de ellos será el mejor
    print("La mejor solucion es: ")
    display(soluciones.head(1))
    
    

In [7]:
display(df_terneros.head(5))
solucionFuerzaBruta(1000)

,Peso,Sexo
1,38.0,H
2,37.1,H
3,35.0,H
4,34.1,H
5,38.0,M



EXPERIMENTO

cantidad de terneros totales:  600
cantidad de terneros por grupo:  66
La mejor solucion es: 


,T1,dT1,T2,dT2,T3,dT3,dTotal
769,"3,1,2",1.07,"7,8,9",1.07,"5,6,4",0.54,2.68


<font color="blue">Calcula la complejidad del algoritmo por fuerza bruta</font>

### Respuesta

La complejidad del algoritmo es O(n^2) ya que la mezcla de grupos se realiza en el primer ciclo "for" y la asignación de Tratamientos a cada grupo se realiza en un "for" interno.

<font color="red">(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta</font>

### Respuesta

Una alternativa a buscar las soluciones de forma aleatoria está en mezclar los grupos en los tratamientos de forma "finita" es decir, las permutaciones de los grupos dentro de cada tratamiento no tiene efecto en mi funcion de homogeneidad, de forma que hay 9 posbles formas de combinar los 9 grupos en 3 tratamientos.


In [8]:
def solucionOptima():

    # ordena la lista y asigna el grupo del 1 al 9 en grupos de igual tamaño
    # el sobrante de individuos se asigna al grupo final
    solucion = dividir_poblacion_9(df_terneros)
    sol_tratamientos = solucion.groupby(["Tratamiento", "Grupo"], as_index=False)["Peso"].agg({"PesoStd": "std", "PesoProm":"mean"})
    display(sol_tratamientos)
    # recolecto los resultados en un dataframe para evaluar la mejor solucion al final de las iteraciones
    soluciones = pd.DataFrame(columns=["T1", "dT1", "T2", "dT2", "T3", "dT3", "dTotal"])
    
    # mover los grupos de forma "circular" en 9 secuencias, una de ellas nos entregará la mejor solución
    # movimiento circular, el primer elemento de la lista va al final y se distribuyen los tratmientos
    # en la siguiente iteracion, el nuevo primer elemento de la lista va al final y se distribuyen los tratamientos en el nuevo orden

    #lista inicial de grupos
    ordenamiento = list(range(1,10))

    for i in range( 1, 10 ) :
        
        for j in ordenamiento:
            tratamiento = int((ordenamiento.index(j) / 3)+1)
            sol_tratamientos.loc[sol_tratamientos["Grupo"]==("Grupo " + str(j)), "Tratamiento"] = "T" + str(tratamiento)
        
        nueva_solucion = pd.DataFrame(columns=["T1", "dT1", "T2", "dT2", "T3", "dT3", "dTotal"], index=[i])
        nueva_solucion.loc[i, "T1"] = ",".join([s.replace('Grupo ' , '') for s in sol_tratamientos[sol_tratamientos["Tratamiento"]=="T1"]["Grupo"].values ])
        nueva_solucion.loc[i,"dT1"] = round( sol_tratamientos[sol_tratamientos["Tratamiento"]=="T1"]["PesoProm"].std(), 2)
        
        nueva_solucion.loc[i, "T2"] = ",".join([s.replace('Grupo ' , '') for s in sol_tratamientos[sol_tratamientos["Tratamiento"]=="T2"]["Grupo"].values ])
        nueva_solucion.loc[i,"dT2"] = round( sol_tratamientos[sol_tratamientos["Tratamiento"]=="T2"]["PesoProm"].std(), 2)
        
        nueva_solucion.loc[i, "T3"] = ",".join([s.replace('Grupo ' , '') for s in sol_tratamientos[sol_tratamientos["Tratamiento"]=="T3"]["Grupo"].values ])
        nueva_solucion.loc[i,"dT3"] = round( sol_tratamientos[sol_tratamientos["Tratamiento"]=="T3"]["PesoProm"].std(), 2)
        
        nueva_solucion.loc[i,"dTotal"] = nueva_solucion.loc[i,"dT1"] + nueva_solucion.loc[i,"dT2"] + nueva_solucion.loc[i,"dT3"]
        soluciones = pd.concat([soluciones, nueva_solucion])
        
        ordenamiento = ordenamiento[-1:] + ordenamiento[:-1]

    #ordenar las soluciones de forma ascendente
    soluciones = soluciones.sort_values( by = ['dTotal'] )
    #al inicio del dataframe estarán las soluciones encontradas, el primero de ellos será el mejor
    print("Las mejores soluciones son las tres primeras filas: ")
    display(soluciones)


In [9]:
solucionOptima()


EXPERIMENTO

cantidad de terneros totales:  600
cantidad de terneros por grupo:  66


,Tratamiento,Grupo,PesoStd,PesoProm
0,,Grupo 1,0.763438,33.619697
1,,Grupo 2,0.237278,34.959091
2,,Grupo 3,0.251842,35.743939
3,,Grupo 4,0.141594,36.431818
4,,Grupo 5,0.154052,36.956061
5,,Grupo 6,0.179640,37.506061
6,,Grupo 7,0.211339,38.131818
7,,Grupo 8,0.261540,38.907576
8,,Grupo 9,0.703312,40.250000


Las mejores soluciones son las tres primeras filas: 


,T1,dT1,T2,dT2,T3,dT3,dTotal
1,"1,2,3",1.07,"4,5,6",0.54,"7,8,9",1.07,2.68
4,"7,8,9",1.07,"1,2,3",1.07,"4,5,6",0.54,2.68
7,"4,5,6",0.54,"7,8,9",1.07,"1,2,3",1.07,2.68
2,"1,2,9",3.51,"3,4,5",0.61,"6,7,8",0.7,4.82
5,"6,7,8",0.7,"1,2,9",3.51,"3,4,5",0.61,4.82
8,"3,4,5",0.61,"6,7,8",0.7,"1,2,9",3.51,4.82
3,"1,8,9",3.51,"2,3,4",0.74,"5,6,7",0.59,4.84
6,"5,6,7",0.59,"1,8,9",3.51,"2,3,4",0.74,4.84
9,"2,3,4",0.74,"5,6,7",0.59,"1,8,9",3.51,4.84


<font color="red">(*)Calcula la complejidad del algoritmo </font>

### Respuesta

El orden sigue siendo cuadrático sin embargo las iteraciones se reducen a 81 en todos los casos.

<font color="blue">Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios</font>

### Respuesta

la funcion "genera_poblacion_aleatoria" realiza este procedimiento

<font color="blue">Aplica el algoritmo al juego de datos generado</font>

### Respuesta

En todo el ejercicio apliqué esta función para obtener los datos para los algoritmos

<font color="blue">Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo</font>

### Respuesta

Con los conocimientos basicos que he adquirido ha sido suficiente, sin embargo todavía recorro mucho a la ayuda de la sintaxis de python y sus funciones

<font color="blue">Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño</font>

### Respuesta

Probé el algoritmo con datos aleatorios de hasta 30,000 y el proceso más pesado es el ordenamiento inicial de los individuos en cada grupo, el calculo de qué grupo va a cada tratamiento es de menor costo.

# Problema 2

## Descripcion

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios que maximice la audiencia.

Los horarios disponibles se conocen a priori y son los siguientes:

Dia | Horario
--- | ---
Viernes | 20
Sabado | 12, 16, 18, 20
Domingo | 12, 16, 18, 20
Lunes | 20

### Restricciones

* En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores que tienen relación directa con la audiencia. Hay 4 equipos en la categoría A, 10 equipos de categoría B y 6 equipos de categoría C.

* Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

. | Categoria A | Categoria B | Categoria C
--- | --- | --- | ---
Categoria A | 2 millones | 1.3 millones | 1 millon
Categoria B | . | 0.9 millones | 0.75 millones
Categoria C | . | . | 0.47 millon

* Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla:

. | Viernes | Sabado | Domingo | Lunes
--- | --- | --- | --- | ---
12 h | - | 0.55 | 0.45 | -
16 h | - | 0.7 | 0.75 | -
18 h | - | 0.8 | 0.85 | -
20 h | 0.4 | 1 | 1 | 0.4

* Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes

* Es posible la coincidencia de horarios pero en este caso la audiencia se verá afectada(no se acumula) y se repartirá según los siguientes criterios:

    - se tomará la mejor audiencia y se repartirá según la siguiente ponderación:
        - A = 4 puntos
        - B = 2 puntos
        - C = 1 punto
        
    - Si hay una coincidencia de horario para dos partidos por ejemplo :
        AB y BC
        Total puntos: 4(A)+2(B) + 2(B)+1(C) =9
        
        La audiencia máxima para ese horario será la de AB, 1.3 Millones
        
        La audiencia (1.3 Millones) se reparte en :
        
        Para AB: $\frac{6}{9} \cdot 1.3 millones = 0.87 millones$
        
        Para BC: $\frac{3}{9} \cdot 1.3 millones = 0.43 millones$
        

<font color="red"> Con estos datos debemos construir un algoritmo que realice la asignación que maximice la audiencia para cualquier jornada de la liga. </font>

<font color="red">(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br></font>

#### Respuesta:

Suponiendo que tengamos "N" equipos, para que cada equipo juegue con todos los demás deben jugarse $N-1$ fechas, para que el mismo equipo no juegue dos veces el mismo dia.

Dentro de cada fecha se pueden jugar "P" partidos, que corresponden combinatoria de los equipos de dos en dos $\binom{N}{2}$

de esos partidos hay que determinar cuantos se pueden jugar en una fecha dada con "H" horarios y corresponde a la combinatoria de "P" tomando "H" partidos por cada fecha.

$$\binom{P}{H}$$

<font color="blue">¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.</font>

#### Respuesta

una condición intrinseca es que un equipo no juegue dos veces el mismo dia, para cumplir con esto, deben existir 19 jornadas distintas, con eso un equipo juega con todos los demás equipos en jornadas diferentes.

La cantidad de partidos está dada por la combinatoria de 10 en 2 $\binom{10}{2} = 190 $

Hay que distribuir esos 190 partidos en las 19 jornadas a razón de 10 partidos por jornada.



<font color="red">
    Modelo para el espacio de soluciones<br>
   
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)
</font>

### Respuesta

Para este problema van a ser de mucha utilidad arreglos, hay algunos parametros que se ajustan perfectamente a esta forma de almacenamiento y las funciones de python para manejar arreglos facilitan su uso.


<font color="red">
Según el modelo para el espacio de soluciones<br>
    
(*)¿Cual es la función objetivo?
</font>

### Respuesta

la función objetivo será maximizar la audiencia en la "plantilla" de partidos de cada jornada


<font color="red">(*)¿Es un problema de maximización o minimización?</font>

### Respuesta

Es un problema de maximización de audciencia

<font color="blue">Diseña un algoritmo para resolver el problema por fuerza bruta</font>

### Respuesta

In [10]:
#arreglo con los equipos: orden, nombre y categoria, partidos
equipos = [('A1','A'), ('A2','A'), ('A3','A'), ('A4','A'), ('B1','B'), ('B2','B'), ('B3','B'), ('B4','B'), ('B5','B'), ('B6','B'), ('B7','B'), ('B8','B'), ('B9','B'), ('B10','B'), ('C1','C'), ('C2','C'), ('C3','C'), ('C4','C'), ('C5','C'), ('C6','C')]
print("Equipos: (nombre, categoria)")
display(equipos)

#diccionario con los factores de audiencia
audiencia = {'AA':2.0, 'AB': 1.3, 'BA':1.3, 'AC':1.0, 'CA':1.0, 'BB':0.9, 'BC':0.75, 'CB':0.75, 'CC':0.47}
print("Factores de audiencia: (categorias, audiencia) en millones")
display(audiencia)

#horarios
horarios = [ ('S:20', 1.0), ('D:20', 1.0), ('D:18', 0.85), ('S:18', 0.8), ('D:16', 0.75), ('S:16', 0.7), ('S:12', 0.55), ('D:12', 0.45), ('V:20', 0.4), ('L:20', 0.4) ]
print("Factores de horarios: (dia:hora, factor)")
display(horarios)


Equipos: (nombre, categoria)


[('A1', 'A'),
 ('A2', 'A'),
 ('A3', 'A'),
 ('A4', 'A'),
 ('B1', 'B'),
 ('B2', 'B'),
 ('B3', 'B'),
 ('B4', 'B'),
 ('B5', 'B'),
 ('B6', 'B'),
 ('B7', 'B'),
 ('B8', 'B'),
 ('B9', 'B'),
 ('B10', 'B'),
 ('C1', 'C'),
 ('C2', 'C'),
 ('C3', 'C'),
 ('C4', 'C'),
 ('C5', 'C'),
 ('C6', 'C')]

Factores de audiencia: (categorias, audiencia) en millones


{'AA': 2.0,
 'AB': 1.3,
 'BA': 1.3,
 'AC': 1.0,
 'CA': 1.0,
 'BB': 0.9,
 'BC': 0.75,
 'CB': 0.75,
 'CC': 0.47}

Factores de horarios: (dia:hora, factor)


[('S:20', 1.0),
 ('D:20', 1.0),
 ('D:18', 0.85),
 ('S:18', 0.8),
 ('D:16', 0.75),
 ('S:16', 0.7),
 ('S:12', 0.55),
 ('D:12', 0.45),
 ('V:20', 0.4),
 ('L:20', 0.4)]

In [11]:
def ligaFuerzaBruta():
    
    #calculo la rotacion posible de los equipos para que no se repitan partidos
    equiposRotacion = np.zeros((19, 20))
    equiposRotacion[:,0] = 1 #el equipo 1 es el pivote para todas las fechas
    for i in range(0, 19):
        for j in range(0, 19):
            equiposRotacion[i:,j+1] = (i+j)%19+2

    #almaceno el resultado de todas las fechas
    jornadas = pd.DataFrame(columns=["partidos", "audienciaBruta", "audienciaHorario"])

    for iJornada in range(0, 19):

        #inicializo las variables de cada jornada
        lista_partido = ""
        audienciaBruta = 0.0
        audienciaHorario = 0.0
        
        for iHorario in range(0, 10):

            #para cada horario tomo los equipos del inicio contra los equipos del final de la lista de equiposRotacion
            lista_partido += " / " + equipos[int(equiposRotacion[iJornada, iHorario])-1][0] + '-' + equipos[int(equiposRotacion[iJornada, 19-iHorario])-1][0]
            
            #calculo la clave de la audiencia
            clave_audiencia = equipos[int(equiposRotacion[iJornada, iHorario])-1][1] + equipos[int(equiposRotacion[iJornada, 19-iHorario])-1][1]
            
            #obtengo el factor del horario
            factor_horario = horarios[iHorario][1]
            audienciaBruta += audiencia[ clave_audiencia ]
            audienciaHorario += ( audiencia[ clave_audiencia ] * factor_horario )
            #print("p", equiposRotacion[iJornada, iHorario], "j", equiposRotacion[iJornada, 19-iHorario], "ab", audienciaBruta, "ah", audienciaHorario)

        #print ("Jornada: ", iJornada, " : ", lista_partido)
        #almaceno el resultado en el dataframe
        nueva_jornada = pd.DataFrame(columns=["partidos", "audienciaBruta", "audienciaHorario"], index=[iJornada])
        nueva_jornada.loc[iJornada, "partidos"]=lista_partido
        nueva_jornada.loc[iJornada, "audienciaBruta"]=audienciaBruta
        nueva_jornada.loc[iJornada, "audienciaHorario"]=audienciaHorario
        jornadas = pd.concat([jornadas, nueva_jornada])

    #partidos
    return jornadas

In [12]:
calendario = ligaFuerzaBruta()
print("audiencia total: ", calendario["audienciaBruta"].sum())
print("audiencia con factores: ", calendario["audienciaHorario"].sum())
calendario.sort_values(by="audienciaHorario", ascending=False)

audiencia total:  180.54999999999998
audiencia con factores:  126.992


,partidos,audienciaBruta,audienciaHorario
2,/ A1-A3 / A4-A2 / B1-C6 / B2-C5 / B3-C4 / B4-...,10.3,7.795
3,/ A1-A4 / B1-A3 / B2-A2 / B3-C6 / B4-C5 / B5-...,10,7.5025
4,/ A1-B1 / B2-A4 / B3-A3 / B4-A2 / B5-C6 / B6-...,9.7,7.1825
5,/ A1-B2 / B3-B1 / B4-A4 / B5-A3 / B6-A2 / B7-...,9.57,7.083
1,/ A1-A2 / A3-C6 / A4-C5 / B1-C4 / B2-C3 / B3-...,9.7,7.075
6,/ A1-B3 / B4-B2 / B5-B1 / B6-A4 / B7-A3 / B8-...,9.44,7.016
7,/ A1-B4 / B5-B3 / B6-B2 / B7-B1 / B8-A4 / B9-...,9.31,6.8725
8,/ A1-B5 / B6-B4 / B7-B3 / B8-B2 / B9-B1 / B10...,9.14,6.646
11,/ A1-B8 / B9-B7 / B10-B6 / C1-B5 / C2-B4 / C3...,9.85,6.6025
12,/ A1-B9 / B10-B8 / C1-B7 / C2-B6 / C3-B5 / C4...,10,6.595


<font color="blue">Calcula la complejidad del algoritmo por fuerza bruta</font>

## Respuesta

La complejidad del algoritmo es cuadrática $2\cdot O^2$ porque para organizar los partidos uso dos ciclos for y para asignar los partidos a cada jornada utilizo otros dos ciclos for.

<font color="red">(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta</font>

### Respuesta

La mejora que propongo es reducir la complejidad a $O^2$, quitando uno de los ciclos for dobles.

In [13]:
def ligaFuerzaBrutaV2():

    #almaceno el resultado de todas las fechas
    jornadas = pd.DataFrame(columns=["partidos", "audienciaBruta", "audienciaHorario"])
    
    #calculo la rotacion posible de los equipos para que no se repitan partidos
    equiposRotacion = np.zeros((19, 20))
    equiposRotacion[:,0] = 1 #el equipo 1 es el pivote para todas las fechas
    
    for i in range(0, 19):
        for j in range(0, 19):
            equiposRotacion[i:,j+1] = (i+j)%19+2
            
        #inicializo las variables de cada jornada
        lista_partido = ""
        audienciaBruta = 0.0
        audienciaHorario = 0.0
        
        for iHorario in range(0, 10):

            #para cada horario tomo los equipos del inicio contra los equipos del final de la lista de equiposRotacion
            lista_partido += " / " + equipos[int(equiposRotacion[i, iHorario])-1][0] + '-' + equipos[int(equiposRotacion[i, 19-iHorario])-1][0]
            
            #calculo la clave de la audiencia
            clave_audiencia = equipos[int(equiposRotacion[i, iHorario])-1][1] + equipos[int(equiposRotacion[i, 19-iHorario])-1][1]
            
            #obtengo el factor del horario
            factor_horario = horarios[iHorario][1]
            audienciaBruta += audiencia[ clave_audiencia ]
            audienciaHorario += ( audiencia[ clave_audiencia ] * factor_horario )
            #print("p", equiposRotacion[iJornada, iHorario], "j", equiposRotacion[iJornada, 19-iHorario], "ab", audienciaBruta, "ah", audienciaHorario)

        #print ("Jornada: ", iJornada, " : ", lista_partido)
        #almaceno el resultado en el dataframe
        nueva_jornada = pd.DataFrame(columns=["partidos", "audienciaBruta", "audienciaHorario"], index=[i])
        nueva_jornada.loc[i, "partidos"]=lista_partido
        nueva_jornada.loc[i, "audienciaBruta"]=audienciaBruta
        nueva_jornada.loc[i, "audienciaHorario"]=audienciaHorario
        jornadas = pd.concat([jornadas, nueva_jornada])


    #partidos
    return jornadas

In [14]:
primeraFecha = ligaFuerzaBrutaV2()
display(primeraFecha)

,partidos,audienciaBruta,audienciaHorario
0,/ A1-C6 / A2-C5 / A3-C4 / A4-C3 / B1-C2 / B2-...,9.1,6.3575
1,/ A1-A2 / A3-C6 / A4-C5 / B1-C4 / B2-C3 / B3-...,9.7,7.075
2,/ A1-A3 / A4-A2 / B1-C6 / B2-C5 / B3-C4 / B4-...,10.3,7.795
3,/ A1-A4 / B1-A3 / B2-A2 / B3-C6 / B4-C5 / B5-...,10,7.5025
4,/ A1-B1 / B2-A4 / B3-A3 / B4-A2 / B5-C6 / B6-...,9.7,7.1825
5,/ A1-B2 / B3-B1 / B4-A4 / B5-A3 / B6-A2 / B7-...,9.57,7.083
6,/ A1-B3 / B4-B2 / B5-B1 / B6-A4 / B7-A3 / B8-...,9.44,7.016
7,/ A1-B4 / B5-B3 / B6-B2 / B7-B1 / B8-A4 / B9-...,9.31,6.8725
8,/ A1-B5 / B6-B4 / B7-B3 / B8-B2 / B9-B1 / B10...,9.14,6.646
9,/ A1-B6 / B7-B5 / B8-B4 / B9-B3 / B10-B2 / C1...,9.12,6.473


<font color="red">(*)Calcula la complejidad del algoritmo </font>

### Respuesta

Sigue siendo cuadrático sin embargo con un factor menor

<font color="blue">Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios</font>

### Respuesta

Se podria "barajar" el orden de los equipos para los partidos, sin embargo la aleatoriedad en los equipos no desempeña un papel importante para el problema.

Pienso que los factores que pueden afectar al problema serian:

- Menor numero de partidos por jornada, esto obligaría a "seleccionar" mejor los equipos para cada partido o podria obligar a jugar varios partidos en la misma jornada

- diferente cantidad de equipos dentro de cada categoria tambien afectaria el problema, cantidades impares por ejemplo, esto nuevamente llevaria a buscar una "combinacion" ideal de partidos para obtener mejores audiencias.

<font color="blue">Aplica el algoritmo al juego de datos generado</font>

Respuesta

<font color="blue">Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo</font>

### Respuesta

Con la resolucion del problema 1, este problema me pareció bastante similar de resolver, so podia haber aplicado un algoritmo voraz para la selección de los horarios de los partidos, pero al ser igual la cantidad de horarios que de partidos por jornada no tiene sentido hacerlo.

Me costó mucho conseguir la "combinatoria" de equipos, pero me sirvió esta referencia en wikipedia de "round robin"

https://en.wikipedia.org/wiki/Round-robin_tournament

<font color="blue">Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño</font>

### Respuesta

Como mencioné en una respuesta anterior, variaciones como cantidades de partidos que sean diferentes a los horarios disponibles, horarios siempre menores que partidos disponibles.  diferentes cantidades de equipos dentro de cada categoria.

La limitacion de que se juegue un viernes y lunes realmente no tiene sentido con las condiciones actuales.

otro avance podria hacer, armar el calendario completo con todas las eliminatorias y optimizar de igual forma la audiencia.  De esta forma los factores de horarios son variables en cada etapa y los factores de audiencia tambien.

# Problema 3

## Descripcion

El problema consiste en analizar el siguiente problema y diseñar un algoritmo que lo resuelva.

Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/)

Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una cantidad dada. Un ejemplo sería para obtener el 4:

4+2-6/3*1 = 4

Debe analizarse el problema para encontrar todos los valores enteros posibles planteando las siguientes cuestiones:

- ¿Qué valor máximo y mínimo a priori se pueden obtener según las condiciones?

- ¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo ?

Nota: Es posible usar la función de python “eval” para evaluar una expresión


<font color="red">(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br></font>

#### Respuesta:

La permutacion de los signos, por la Permutacion de los 9 digitos en grupos de la cantidad de signos permutados mas 1:

$$ signos ! \cdot \sum _{i = signos +1}^9 P \binom{9 } {i } $$

<font color="blue">¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.</font>

#### Respuesta

es la permutacion de los signos, por la permutacion de los 9 digitos en grupos de 5

$$4 ! \cdot P \binom {9} {5} $$


<font color="red">Modelo para el espacio de soluciones<br>
    
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)
</font>

### Respuesta

Con los arreglos de pythons es suficiente para organizar las soluciones de este problema

<font color="red">
Según el modelo para el espacio de soluciones<br>
    
(*)¿Cual es la función objetivo?
</font>

### Respuesta

La función objetivo es obtener una cantidad entera aplicando 4 operaciones a 5 digitos

Ejemplo:

    9 * 8 / 1 + 7 - 2 = 77 -- maxima solucion entera
    
    1 - 9 * 8 + 4 / 2 = -69 -- minima solucion entera
    
    9 * 8 / 7 + 6 - 5 = 11.28 -- no es solucion valida

<font color="red">(*)¿Es un problema de maximización o minimización?</font>

### Respuesta

Es un problema de programacion lineal 

<font color="blue">Diseña un algoritmo para resolver el problema por fuerza bruta</font>

### Respuesta

In [15]:
def digitosFuerzaBruta():
    operadores= [ '+', '-', '*', '/' ]
    resultados = pd.DataFrame(columns=["operacion", "resultado"])
    operacion=[ '0', '', '0', '', '0', '', '0', '', '0' ]
    print("Inicio: ", datetime.now())
    for i in range( 1, 10 ) :
        operacion[0] = str(i)
        
        for w in range(0, 4) : #primer signo
            operacion[1] = operadores[w]

            for j in range( 1, 10 ) :
                if i != j:
                    operacion[2] = str(j)
                    
                    for x in range(0, 4) : #segundo signo
                        if w != x:
                            operacion[3] = operadores[x]

                            for k in range(1, 10) :
                                if (i!=k) & (j!=k) :
                                    operacion[4] = str(k)
                    
                                    for y in range(0, 4) : #tercer signo
                                        if (w != y) & (x != y):
                                            operacion[5] = operadores[y]

                                            for l in range(1, 10) :
                                                if (i!=l) & (j!=l) & (k!=l) :
                                                    operacion[6] = str(l)
                    
                                                    for z in range(0, 4) : #cuarto signo
                                                        if (w != z) & (x != z) & (y != z):
                                                            operacion[7] = operadores[z]

                                                            for m in range(1, 10) :
                                                                if (i!=m) & (j!=m) & (k!=m) & (l!=m) :
                                                                    operacion[8] = str(m)
                                                                    str_operacion = "".join(operacion)
                                                                    valor = eval(str_operacion)
                                                                    if(valor == int(valor)):
                                                                        nuevo_resultado = pd.DataFrame(columns=["operacion", "resultado"], data=[[str_operacion, valor]])
                                                                        resultados = pd.concat([resultados, nuevo_resultado])
        print("digito: ", i, "  |  resultados: ", resultados.shape[0])

    print("Fin: ", datetime.now())
    return resultados;

In [16]:
resDigitosFB = digitosFuerzaBruta()


Inicio:  2019-03-11 15:47:43.791140
digito:  1   |  resultados:  5520
digito:  2   |  resultados:  13960
digito:  3   |  resultados:  23080
digito:  4   |  resultados:  33640
digito:  5   |  resultados:  44200
digito:  6   |  resultados:  55960
digito:  7   |  resultados:  66520
digito:  8   |  resultados:  78600
digito:  9   |  resultados:  90000
Fin:  2019-03-11 15:51:19.306355


In [17]:
resDigitosFB.sort_values(by="resultado")


,operacion,resultado
0,6/3-8*9+1,-69.0
0,1-9*8+4/2,-69.0
0,1-9*8+6/3,-69.0
0,1-8*9+6/3,-69.0
0,1-8*9+4/2,-69.0
0,1+4/2-8*9,-69.0
0,1+4/2-9*8,-69.0
0,6/3+1-8*9,-69.0
0,1+6/3-9*8,-69.0
0,1+6/3-8*9,-69.0


<font color="blue">Calcula la complejidad del algoritmo por fuerza bruta</font>

### Respuesta

La complejidad es de orden nueve $O^9$, nueve ciclos for anidados, son 362880 combinaciones posibles, no todas ellas arrojan un resultado entero.

la cantidad de operaciones que arrojan un resultado entero son: 90000


<font color="red">(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta</font>

Respuesta

In [18]:
# si bien voy a utilizar los mismos ciclos for, esta ves voy a agregar condiciones para descartar combinaciones inválidad
def digitosFuerzaBrutaMejorado():
    operadores= [ '+', '-', '*', '/' ]
    resultados = pd.DataFrame(columns=["operacion", "resultado"])
    operacion=[ '0', '', '0', '', '0', '', '0', '', '0' ]
    continuar = True
    operacion_temp = ""
    valor_temp = 0.0
    operacion_final = ""
    valor_final = 0.0
    
    print("Inicio: ", datetime.now())
    for i in range( 1, 10 ) :
        operacion[0] = str(i)
        
        for w in range(0, 4) : #primer signo
            operacion[1] = operadores[w]

            for j in range( 1, 10 ) :                
                if (i != j):
                    operacion[2] = str(j)
                    
                    if(operacion[1] == "/"):
                        #evaluacion temporal de la primera operacion parcial
                        operacion_temp = "".join(operacion[:3])
                        valor_temp = eval(operacion_temp)
                        continuar = valor_temp == int(valor_temp)
                    else:
                        continuar = True
                        
                    if(continuar):
                        for x in range(0, 4) : #segundo signo
                            if w != x:
                                operacion[3] = operadores[x]

                                for k in range(1, 10) :
                                    if (i!=k) & (j!=k) :
                                        operacion[4] = str(k)
                                        
                                        if(operacion[3] == "/"):
                                            #evaluacion temporal de la primera operacion parcial
                                            operacion_temp = "".join(operacion[:5])
                                            valor_temp = eval(operacion_temp)
                                            continuar = valor_temp == int(valor_temp)
                                        else:
                                            continuar = True

                                        if(continuar):
                                            for y in range(0, 4) : #tercer signo
                                                if (w != y) & (x != y):
                                                    operacion[5] = operadores[y]

                                                    for l in range(1, 10) :
                                                        if (i!=l) & (j!=l) & (k!=l) :
                                                            operacion[6] = str(l)

                                                            if(operacion[5] == "/"):
                                                                #evaluacion temporal de la primera operacion parcial
                                                                operacion_temp = "".join(operacion[:7])
                                                                valor_temp = eval(operacion_temp)
                                                                continuar = valor_temp == int(valor_temp)
                                                            else:
                                                                continuar = True

                                                            if(continuar):
                                                                for z in range(0, 4) : #cuarto signo
                                                                    if (w != z) & (x != z) & (y != z):
                                                                        operacion[7] = operadores[z]

                                                                        for m in range(1, 10) :
                                                                            if (i!=m) & (j!=m) & (k!=m) & (l!=m) :
                                                                                operacion[8] = str(m)
                                                                                operacion_final = "".join(operacion)
                                                                                valor_final = eval(operacion_final)
                                                                                if(valor_final == int(valor_final)):
                                                                                    nuevo_resultado = pd.DataFrame(columns=["operacion", "resultado"], data=[[operacion_final, valor_final]])
                                                                                    resultados = pd.concat([resultados, nuevo_resultado])
        print("digito: ", i, "  |  resultados: ", resultados.shape[0])

    print("Fin: ", datetime.now())
    return resultados;

In [19]:
resDigitosFBM = digitosFuerzaBrutaMejorado()

Inicio:  2019-03-11 15:51:19.389003
digito:  1   |  resultados:  4200
digito:  2   |  resultados:  11660
digito:  3   |  resultados:  19700
digito:  4   |  resultados:  29300
digito:  5   |  resultados:  38540
digito:  6   |  resultados:  49580
digito:  7   |  resultados:  58820
digito:  8   |  resultados:  70020
digito:  9   |  resultados:  80280
Fin:  2019-03-11 15:54:16.689477


In [20]:
resDigitosFBM.sort_values(by="resultado")

,operacion,resultado
0,1+6/3-9*8,-69.0
0,6/3+1-9*8,-69.0
0,1-8*9+4/2,-69.0
0,4/2-9*8+1,-69.0
0,6/3-8*9+1,-69.0
0,1+6/3-8*9,-69.0
0,1-9*8+4/2,-69.0
0,1-8*9+6/3,-69.0
0,4/2-8*9+1,-69.0
0,6/3-9*8+1,-69.0


<font color="red">(*)Calcula la complejidad del algoritmo </font>

### Respuesta

La complejidad del algoritmo sigue siendo de noveno orden, sin embargo se realizan menos operaciones, inclusive se encuentran menos soluciones ya que se descartan algunas "inviables" a priori.

Por ejemplo descartaria una solucion cuya primera operacion sea 5 / 2 porque es un resultado decimal, a pesar que su siguiente operacion sea * 4 que lo volvería entero nuevamente.

<font color="blue">Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios</font>

### Respuesta

No pienso que tenga sentido tener entradas aleatorias dada las limitaciones que tenemos en juego

<font color="blue">Aplica el algoritmo al juego de datos generado</font>

Respuesta

<font color="blue">Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo</font>

Respuesta

<font color="blue">Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño</font>

Respuesta